<a href="https://colab.research.google.com/github/lamtranBKHN/PCB-defect-detector/blob/master/PCB_defect_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from matplotlib import pyplot as plt
import cv2
import pickle

In [0]:
IMG_SIZE = 224

file_list = []
class_list = []

DATADIR = "Organized dataSet"

# All the categories neural network will detect
CATEGORIES = ["mouse bite","not","open","pin hole",
              "short","spur","spurious copper"]

# Checking or all images in the data folder
for category in CATEGORIES :
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)

training_data = []

def create_training_data():
    for category in CATEGORIES :
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()
random.shuffle(training_data)

X = [] #features
y = [] #labels

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)   
# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

# Opening the files about data
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0

In [0]:
#from keras.utils import to_categorical
#y = to_categorical(y)
# Load model VGG 16 
baseModel = VGG16(weights='imagenet', include_top=False, \
                  input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
# Buil layer
fcHead = baseModel.output
# Flatten 
fcHead = Flatten()(fcHead)
# Add FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.3)(fcHead)
# Output layer with softmax activation
fcHead = Dense(7, activation='softmax')(fcHead)
# modle
model = model = Model(inputs=baseModel.input, outputs=fcHead)

In [0]:
model.compile(loss = "sparse_categorical_crossentropy", 
                    optimizer = SGD(lr=1e-5, momentum=0.9), 
                    metrics=["accuracy"])
# Training the model.
X = np.array(X)
y = np.array(y)
history = model.fit(X, y, batch_size=32, epochs=150, validation_split=0.1)

# Saving the model
model_json = model.to_json()
with open("model.json", "w") as json_file :
    json_file.write(model_json)

model.save_weights("model2.h5")
print("Saved model to disk")

model.save('CNN2.model')

# Printing a graph showing the accuracy changes during the training phase
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Train on 1863 samples, validate on 208 samples
Epoch 1/500
1863/1863 [==============================] - 15s 8ms/step - loss: 1.6948 - accuracy: 0.4321 - val_loss: 1.3474 - val_accuracy: 0.5769
Epoch 2/500
1863/1863 [==============================] - 14s 8ms/step - loss: 1.4500 - accuracy: 0.5169 - val_loss: 1.2494 - val_accuracy: 0.5769
Epoch 3/500
1863/1863 [==============================] - 14s 8ms/step - loss: 1.3772 - accuracy: 0.5309 - val_loss: 1.1597 - val_accuracy: 0.5769
Epoch 4/500
1863/1863 [==============================] - 14s 8ms/step - loss: 1.2818 - accuracy: 0.5631 - val_loss: 1.0734 - val_accuracy: 0.6346
Epoch 5/500
1863/1863 [==============================] - 14s 8ms/step - loss: 1.1815 - accuracy: 0.6087 - val_loss: 0.9723 - val_accuracy: 0.6587
Epoch 6/500
1863/1863 [==============================] - 14s 8ms/step - loss: 1.0636 - accuracy: 0.6522 - val_loss: 0.8732 - val_accuracy: 0.7356
Epoch 7/500
1863/1863 [==============================] - 14s 8ms/step - loss:

KeyboardInterrupt: ignored